In [1]:
import os
import sys
sys.path.append('../va_pipeline/')

In [2]:
import subprocess
import pandas as pd
import numpy as np

from calculate_accuracy import *
from drawbox import *
from sensor import exec_file


In [ ]:
draw_boxes(
    video_path='../samples/sparse.mp4', 
    ground_box=pd.read_csv('./test_results/config_testing/sparse_yolov5l_ground_truth.csv'),
    inference_box=pd.read_csv('./test_results/config_testing/frame_diff/sparse/diff_sparse_yolov5n_1280_720_25fps_inference.csv'),
    output_path='../samples/output_video.mp4')